In [57]:
import numpy as np
import pandas as pd
import json
import random

PRS_orig_feature_matrix = np.load('./PRS_feature_matrix_only_ad.npy').astype(np.float32)
usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
final_samples_6yrs=json.load(open('./Final_Samples_6yrs.json'))
# final_samples_2yrs=json.load(open('./Final_Samples_2yrs.json'))
covar_df = pd.read_csv('./COVAR_FILE_bigger_dataset.txt', ' ') 
covar_df['AGE'] = covar_df['AGE'] / 100.0

print(usable_samples_ADNI.__len__())
print(final_samples_6yrs.__len__())
# print(final_samples_2yrs.__len__())
print(PRS_orig_feature_matrix.shape)
print(len(final_samples_6yrs))



1816
974
(1816,)
974


In [58]:
# cnt = number of missing IDs for which covar data doesn't exist
cnt = 0
print("Before adding covar_df columns, shape : ",PRS_orig_feature_matrix.shape)
# adding ( total columns - 2 ) of covar_df , excluding FID, IID
FEATURE_MATRIX = np.concatenate([PRS_orig_feature_matrix.reshape(PRS_orig_feature_matrix.shape[0],1), np.zeros([PRS_orig_feature_matrix.shape[0], 1 ])],1).astype(np.float32)
print("Before adding covar_df columns, shape : ",FEATURE_MATRIX.shape)
for sample in usable_samples_ADNI:
    # taking from the PCs, skipping the first two columns of IID, FID
    age = covar_df[covar_df['IID'] == sample]['AGE']
    # shape[0] = 1 means a row is found in covar for the following sample ID
    # if not, that means no covar data exists for the sample in usable_samples_ADNI
#     print(usable_samples_ADNI[sample],age)
#     print(FEATURE_MATRIX[usable_samples_ADNI[sample]])
    if age.shape[0] != 1:
#         print(covar.shape)
#         print(sample)
        cnt += 1
        continue
    # Adding the covar values to the feature matrix
    FEATURE_MATRIX[usable_samples_ADNI[sample], -1] = age
#     print(FEATURE_MATRIX[usable_samples_ADNI[sample]])


print("Count of missing samples for covar data : ", cnt)
#     FEATURE_MATRIX[usable_samples_ADNI[sample], num_features:] = covar # naeem's modification
# cnt/FEATURE_MATRIX.shape[0], FEATURE_MATRIX[:2], PRS_feature_matrix[:2]

Before adding covar_df columns, shape :  (1816,)
Before adding covar_df columns, shape :  (1816, 2)
Count of missing samples for covar data :  234


In [59]:

# removing age zero indices
# FEATURE_MATRIX = np.delete(FEATURE_MATRIX, age_zero_idx, axis = 0)
print(FEATURE_MATRIX.shape)

(1816, 2)


In [60]:
usable_indices = [( usable_samples_ADNI[final_samples_6yrs[i][0]] if ( final_samples_6yrs[i][0] in usable_samples_ADNI.keys() ) else None ) for i in range(len(final_samples_6yrs))]
print(len(usable_indices))
# print(usable_indices)
FEATURE_MATRIX = FEATURE_MATRIX[usable_indices]
age_zero = 0
age_zero_idx = []
for i in range( len(FEATURE_MATRIX) ):
    if FEATURE_MATRIX[i, -1] == 0.00:
        age_zero += 1
        age_zero_idx.append(i)
        
print(age_zero)
FEATURE_MATRIX = np.delete(FEATURE_MATRIX, age_zero_idx, axis = 0)
# usable_features = np.delete(usable_features, age_zero_idx, axis = 0)
final_samples_6yrs = np.delete(final_samples_6yrs, age_zero_idx, axis = 0)
# usable_samples_ADNI
# usable_samples_ADNI = np.delete(usable_samples_ADNI, age_zero_idx, axis = 0)


print("Shape of feature matrix : ", usable_samples_ADNI.shape)


974
166


AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
print(usable_samples_ADNI)

In [ ]:
# print( df['output'].value_counts() )

# ones = df[df['output'] == 1]
# zeros = df[df['output'] == 0]
# min_len = min( len(ones), len(zeros) ) 

# ones = ones.iloc[:min_len, :]
# zeros = zeros.iloc[:min_len, :]

# df = ones.append(zeros, ignore_index=True)
# print(df.shape)
print(final_samples_6yrs[1][1]==1)

In [ ]:
# print(final_samples_6yrs)
positive_samples = []
negative_samples = []

final_positive_samples = []
final_negative_samples = []
for i,s in enumerate(final_samples_6yrs):
    if s[1] == '0':
        negative_samples.append(FEATURE_MATRIX[i])
        final_negative_samples.append(s)
    else:
        positive_samples.append(FEATURE_MATRIX[i])
        final_positive_samples.append(s)

min_len=min(len(positive_samples),len(negative_samples))

# np.shuffle(final_positive_samples)
# np.shuffle(final_negative_samples)
final_samples_reduced=final_positive_samples[:min_len]+final_negative_samples[:min_len]
total_prs_samples=negative_samples[:min_len]+positive_samples[:min_len]

print(len(final_samples_reduced))


    
    

In [ ]:


# # random.shuffle(final_samples_6yrs)

# for i,sample in enumerate(usable_samples_ADNI):
#     for s in final_samples_6yrs:
#         # print(s)
#         if sample == s[0]:
            
# # print(PRS_to_AD_mapping.__len__())

In [ ]:
# import matplotlib.pyplot as plt
# print("Positive samples mean PRS :",np.mean(positive_samples),"standard deviation",np.std(positive_samples))
# plt.hist(positive_samples,10)
# plt.show()
# plt.hist(negative_samples,10)
# print("Negative samples mean PRS :",np.mean(negative_samples),"standard deviation",np.std(negative_samples))

In [ ]:
# # making 0 as threshold
# y_original=[]
# y_predicted=[]

# for i,sample in enumerate(usable_samples_ADNI):
    
#     for s in final_samples_6yrs:
#         # print(s)
#         if sample == s[0]: 
#             y_predicted.append(PRS_orig_feature_matrix[i] >= 0)
#             y_original.append(s[1])
      


In [ ]:
# from sklearn import metrics
# true=0
# for i in range(y_original.__len__()):
#     true+=(y_original[i]==y_predicted[i])
# precision, recall, fscore,support = metrics.precision_recall_fscore_support(y_original, y_predicted,average="macro")
# AUROC=metrics.roc_auc_score(y_original,y_predicted)
# p, r, thresholds = metrics.precision_recall_curve(y_original,y_predicted)
# AUPRC = metrics.auc(r, p)
# print("precision: ",precision,"recall: ", recall,"f1 score:", fscore)
# print("accuracy",true/(y_original.__len__()))
# print("AUROC:",AUROC,"AUPRC:",AUPRC)

In [ ]:
# # making 0.6 as threshold
# y_original=[]
# y_predicted=[]

# for i,sample in enumerate(usable_samples_ADNI):
    
#     for s in final_samples_6yrs:
#         # print(s)
#         if sample == s[0]: 
#             y_predicted.append(PRS_orig_feature_matrix[i] >= 0.6)
#             y_original.append(s[1])
      


In [ ]:
# from sklearn import metrics
# true=0
# # for i in range(y_original.__len__()):
# #     true+=(y_original[i]==y_predicted[i])
# precision, recall, fscore,support = metrics.precision_recall_fscore_support(y_original, y_predicted,average="macro")
# AUROC=metrics.roc_auc_score(y_original,y_predicted)
# p, r, thresholds = metrics.precision_recall_curve(y_original,y_predicted)
# AUPRC = metrics.auc(r, p)
# print("precision: ",precision,"recall: ", recall,"f1 score:", fscore)
# print("accuracy",true/(y_original.__len__()))
# print("AUROC:",AUROC,"AUPRC:",AUPRC)

In [ ]:
# print(len(positive_samples))

In [ ]:
# import random
# random.shuffle(positive_samples)
# positive_samples_reduced=positive_samples[:len(negative_samples)]
# total_samples=positive_samples+negative_samples
# print(len(negative_samples),len(total_samples))

In [ ]:
mean_of_total=np.mean(total_prs_samples)
median_of_total=np.median(total_prs_samples)

In [ ]:
print("mean",mean_of_total)
print("median",median_of_total)

In [61]:
# making mean as threshold
y_original=[]
y_predicted=[]

for i,sample in enumerate(usable_samples_ADNI):
    
    for s in final_samples_reduced:
        # print(s)
        if sample == s[0]: 
            y_predicted.append(PRS_orig_feature_matrix[i] >= mean_of_total)
            y_original.append(s[1])
      


In [65]:

for i in range(len(y_original)):
    y_original[i]=int(y_original[i])
print(y_original)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 

In [67]:
from sklearn import metrics
true=0
for i in range(y_original.__len__()):
    true+=(y_original[i]==y_predicted[i])
precision, recall, fscore,support = metrics.precision_recall_fscore_support(y_original, y_predicted,average="macro")
AUROC=metrics.roc_auc_score(y_original,y_predicted)
p, r, thresholds = metrics.precision_recall_curve(y_original,y_predicted)
AUPRC = metrics.auc(r, p)
print("precision: ",precision,"recall: ", recall,"f1 score:", fscore)
print("accuracy",true/(y_original.__len__()))
print("AUROC:",AUROC,"AUPRC:",AUPRC)

precision:  0.4503495505778285 recall:  0.4547413793103448 f1 score: 0.4424110005462274
accuracy 0.4547413793103448
AUROC: 0.4547413793103448 AUPRC: 0.5443000317325999


In [70]:
# making median as threshold
y_original=[]
y_predicted=[]

for i,sample in enumerate(usable_samples_ADNI):
    
    for s in final_samples_reduced:
        # print(s)
        if sample == s[0]: 
            y_predicted.append(PRS_orig_feature_matrix[i] >= median_of_total)
            y_original.append(s[1])

for i in range(len(y_original)):
    y_original[i]=int(y_original[i])
print(y_original)
      


[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 

In [71]:
from sklearn import metrics
true=0
for i in range(y_original.__len__()):
    true+=(y_original[i]==y_predicted[i])
precision, recall, fscore,support = metrics.precision_recall_fscore_support(y_original, y_predicted,average="macro")
AUROC=metrics.roc_auc_score(y_original,y_predicted)
p, r, thresholds = metrics.precision_recall_curve(y_original,y_predicted)
AUPRC = metrics.auc(r, p)
print("precision: ",precision,"recall: ", recall,"f1 score:", fscore)
print("accuracy",true/(y_original.__len__()))
print("AUROC:",AUROC,"AUPRC:",AUPRC)

precision:  0.41158536585365857 recall:  0.4267241379310345 f1 score: 0.4010869565217391
accuracy 0.4267241379310345
AUROC: 0.4267241379310345 AUPRC: 0.49245689655172414
